In [1]:
# Dependencies
import requests
import json
import quandl #https://docs.quandl.com/docs/python-time-series give a how to guide
from config import q_api_key
from config import y_api_key
from config import c_api_key

import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
c = Census(c_api_key, year=2017)

import gmaps
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

from uszipcode import SearchEngine, SimpleZipcode, Zipcode #https://uszipcode.readthedocs.io/index.html
#create a variable for the searchengine function which pulls down all lat/longs in database for crime/airbnb data (lat/long to zip)
search = SearchEngine()

In [2]:
#use this call to set the api_key for zillow data
quandl.ApiConfig.api_key = q_api_key

In [3]:
#pull in zipcode list
zipcode_df=pd.read_csv("Resources/zipcode.csv")
zipcode_df['Zipcode'] = pd.to_numeric(zipcode_df['Zipcode'])
zipcode_df.head()

,Zipcode
0,60601
1,60602
2,60603
3,60604
4,60605


In [4]:
#turn zipcode into a list
zipcodes = zipcode_df['Zipcode'].tolist()

#ZILLOW DATA
#create a blank list for median home values and median rent from Zillow
medianrent=[]
medianhomevalues=[]
#pull median home value for each zip code in Chicago at the yearly level for 2018 only
for x in range(0,len(zipcodes)):
    try:
        mr_raw=quandl.get(f"ZILLOW/Z{zipcodes[x]}_ZRIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mr_list=mr_raw["Value"].tolist()
        medianrent.append(mr_list[0])
        print(f'Pulled rent data for {zipcodes[x]}')
    except:
        mr_missing='NaN'
        medianrent.append(mr_missing)
        print(f'Rent data for {zipcodes[x]} not found')
    try:
        mhv_raw=quandl.get(f"ZILLOW/Z{zipcodes[x]}_ZHVIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mhv_list=mhv_raw["Value"].tolist()
        medianhomevalues.append(mhv_list[0])
        print(f'Pulled home value data for {zipcodes[x]}')
    except:
        mhv_missing='NaN'
        medianhomevalues.append(mhv_missing)
        print(f'Home value data for {zipcodes[x]} not found')
print(medianrent)
print(medianhomevalues)

Pulled rent data for 60601
Pulled home value data for 60601
Pulled rent data for 60602
Home value data for 60602 not found
Pulled rent data for 60603
Home value data for 60603 not found
Pulled rent data for 60604
Home value data for 60604 not found
Pulled rent data for 60605
Pulled home value data for 60605
Pulled rent data for 60606
Pulled home value data for 60606
Pulled rent data for 60607
Pulled home value data for 60607
Pulled rent data for 60608
Pulled home value data for 60608
Pulled rent data for 60609
Pulled home value data for 60609
Pulled rent data for 60610
Pulled home value data for 60610
Pulled rent data for 60611
Pulled home value data for 60611
Pulled rent data for 60612
Pulled home value data for 60612
Pulled rent data for 60613
Pulled home value data for 60613
Pulled rent data for 60614
Pulled home value data for 60614
Pulled rent data for 60615
Pulled home value data for 60615
Pulled rent data for 60616
Pulled home value data for 60616
Pulled rent data for 60617
Home

In [5]:
#create a dataframe of zipcode, median rent, and median home values
housing_df = pd.DataFrame({"Zipcode":zipcodes, "Median Rent":medianrent, "Median Home Value":medianhomevalues})
housing_df.head()

,Zipcode,Median Rent,Median Home Value
0,60601,1944.0,391000
1,60602,1666.0,NaN
2,60603,1630.0,NaN
3,60604,1835.0,NaN
4,60605,1865.0,284200


In [6]:
# Extract zipcode and median rent data from the housing data

rent_df = housing_df[["Zipcode", "Median Rent"]]
rent_df.head()

,Zipcode,Median Rent
0,60601,1944.0
1,60602,1666.0
2,60603,1630.0
3,60604,1835.0
4,60605,1865.0


In [7]:
#YELP DATA
#found in https://python.gotrained.com/yelp-fusion-api-tutorial/
headers = {'Authorization': 'Bearer %s' % y_api_key}

#category of business you want to pull 1. nightlife
categories = 'nightlife' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_l = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_l.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

starting datapull for 60601
finished datapull for 60601
new appended category total: 48  for 60601
starting datapull for 60602
finished datapull for 60602
new appended category total: 11  for 60602
starting datapull for 60603
finished datapull for 60603
new appended category total: 22  for 60603
starting datapull for 60604
finished datapull for 60604
new appended category total: 5  for 60604
starting datapull for 60605
finished datapull for 60605
new appended category total: 34  for 60605
starting datapull for 60606
finished datapull for 60606
new appended category total: 22  for 60606
starting datapull for 60607
finished datapull for 60607
new appended category total: 81  for 60607
starting datapull for 60608
finished datapull for 60608
new appended category total: 34  for 60608
starting datapull for 60609
finished datapull for 60609
new appended category total: 10  for 60609
starting datapull for 60610
finished datapull for 60610
new appended category total: 71  for 60610
starting da

In [8]:
#category of business you want to pull 2. restaurants
categories = 'restaurants' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_r = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_r.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

starting datapull for 60601
finished datapull for 60601
new appended category total: 150  for 60601
starting datapull for 60602
finished datapull for 60602
new appended category total: 78  for 60602
starting datapull for 60603
finished datapull for 60603
new appended category total: 96  for 60603
starting datapull for 60604
finished datapull for 60604
new appended category total: 41  for 60604
starting datapull for 60605
finished datapull for 60605
new appended category total: 112  for 60605
starting datapull for 60606
finished datapull for 60606
new appended category total: 163  for 60606
starting datapull for 60607
finished datapull for 60607
new appended category total: 221  for 60607
starting datapull for 60608
finished datapull for 60608
new appended category total: 151  for 60608
starting datapull for 60609
finished datapull for 60609
new appended category total: 90  for 60609
starting datapull for 60610
finished datapull for 60610
new appended category total: 131  for 60610
star

In [9]:
#category of business you want to pull 3. grocery
categories = 'grocery' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_g = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_g.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

starting datapull for 60601
finished datapull for 60601
new appended category total: 7  for 60601
starting datapull for 60602
finished datapull for 60602
new appended category total: 0  for 60602
starting datapull for 60603
finished datapull for 60603
new appended category total: 1  for 60603
starting datapull for 60604
finished datapull for 60604
new appended category total: 0  for 60604
starting datapull for 60605
finished datapull for 60605
new appended category total: 7  for 60605
starting datapull for 60606
finished datapull for 60606
new appended category total: 3  for 60606
starting datapull for 60607
finished datapull for 60607
new appended category total: 6  for 60607
starting datapull for 60608
finished datapull for 60608
new appended category total: 22  for 60608
starting datapull for 60609
finished datapull for 60609
new appended category total: 8  for 60609
starting datapull for 60610
finished datapull for 60610
new appended category total: 11  for 60610
starting datapull 

In [40]:
#combine nightlife, restaurant and grocery with zipcode into dataframe
yelp_df=pd.DataFrame({"Zipcode":zipcodes, "Total Nightlife": category_l, "Total Restaurant": category_r, 
                      "Total Grocery": category_g})

In [41]:
yelp_df.to_csv("Output/Yelp_Data.csv", encoding="utf-8", index=False)

In [7]:
#CRIME DATA
#read in crime data
df_crime = pd.read_csv('Resources/Crimes_2018.csv')
df_crime_latlon=df_crime.loc[:,['Latitude', 'Longitude']]
df_crime_latlon.head()

,Latitude,Longitude
0,41.689079,-87.696064
1,41.763181,-87.657709
2,41.740521,-87.647391
3,41.857068,-87.657625
4,41.751914,-87.647717


In [8]:
len(df_crime_latlon)

263455

In [9]:
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()

In [10]:
res = []
#for every row in df_crime_latlon
for index, row in df_crime_latlon.iterrows():
    #part of uszipcode library it takes the lat/lon from the row you are in and looks for the nearest zip (returns=1) 
    #in a 3 mile radius
    result = search.by_coordinates(row["Latitude"], row["Longitude"], radius=3, returns=1)
    #append to res
    res.append(result)

In [11]:
#create a blank list for zipcodes
zipcodes_crime = []
#for every entry returned in res
for i in range(len(res)):
    #this is a list of lists so for every entry there is another list inside that contains geographical information
    for x in res[i]:
        # zipcode is from uszipcode library and find the zipcode in the list
        zip = x.zipcode
     #   first = res[i]
        #append the zipcode to the zipcodes list
        zipcodes_crime.append(zip)
#     zipcodes.append(first)
len(zipcodes_crime)

263455

In [12]:
#create a dataframe of zipcodes
crime_zip_df = pd.DataFrame({"Zipcode": zipcodes_crime})

In [13]:
#count number of zip codes and reset index so zip code becomes a column
zipcode_count = crime_zip_df["Zipcode"].value_counts().reset_index()
#rename the variables and output to csv
crime_count = zipcode_count.rename(columns={"index": "Zipcode","Zipcode":"Crime Count"})

#make sure zipcode is numeric
crime_count['Zipcode'] = pd.to_numeric(crime_count['Zipcode'])
crime_count_df = crime_count.sort_values("Zipcode")
crime_count_df

,Zipcode,Crime Count
67,46327,135
64,46394,205
52,60018,1004
80,60068,1
77,60106,12
72,60131,41
76,60171,14
68,60176,127
79,60202,4
54,60302,794


In [14]:
#export data to csv
crime_count_df.to_csv("Output/Crime_Count_by_Zip.csv", encoding="utf-8", index=False)

CENSUS DATA

In [7]:
# CENSUS DATA

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})



# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

   

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]
census_pd['Zipcode'] = pd.to_numeric(census_pd['Zipcode'])
census_pd.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915
1,602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283
2,603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416
3,606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168
4,610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369


In [8]:
chicago_census_df = pd.merge(zipcode_df, census_pd, on="Zipcode")
chicago_census_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,60601,13887.0,37.6,110215.0,95152.0,1171.0,8.432347
1,60602,1277.0,31.1,115774.0,77279.0,32.0,2.505873
2,60603,1197.0,26.7,140114.0,106207.0,261.0,21.804511
3,60604,668.0,40.6,114565.0,130966.0,127.0,19.011976
4,60605,26188.0,33.6,107811.0,81144.0,2312.0,8.828471


In [9]:
chicago_census_df["Lat"] = ""
chicago_census_df["Lng"] = ""
chicago_census_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Lat,Lng
0,60601,13887.0,37.6,110215.0,95152.0,1171.0,8.432347,,
1,60602,1277.0,31.1,115774.0,77279.0,32.0,2.505873,,
2,60603,1197.0,26.7,140114.0,106207.0,261.0,21.804511,,
3,60604,668.0,40.6,114565.0,130966.0,127.0,19.011976,,
4,60605,26188.0,33.6,107811.0,81144.0,2312.0,8.828471,,


In [10]:
params = {"key": gkey}

for index, row in chicago_census_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    address = row['Zipcode']

    # update address key value
    params['address'] = f"{address}"

    # make request
    response = requests.get(base_url, params=params).json()
    
  
    # convert to json
#     zip_lat_lng = response.json()

    chicago_census_df.loc[index, "Lat"] = response["results"][0]["geometry"]["location"]["lat"]
    chicago_census_df.loc[index, "Lng"] = response["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
chicago_census_df.head()
# zip_lat_lng

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Lat,Lng
0,60601,13887.0,37.6,110215.0,95152.0,1171.0,8.432347,41.884,-87.6197
1,60602,1277.0,31.1,115774.0,77279.0,32.0,2.505873,41.8835,-87.6291
2,60603,1197.0,26.7,140114.0,106207.0,261.0,21.804511,41.8809,-87.6254
3,60604,668.0,40.6,114565.0,130966.0,127.0,19.011976,41.8785,-87.6279
4,60605,26188.0,33.6,107811.0,81144.0,2312.0,8.828471,41.8703,-87.6236


In [11]:
# Merge housing data into the census data

chicago_housing_census_df =  pd.merge(housing_df, chicago_census_df, on="Zipcode")
chicago_housing_census_df.head()

,Zipcode,Median Rent,Median Home Value,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Lat,Lng
0,60601,1944.0,391000,13887.0,37.6,110215.0,95152.0,1171.0,8.432347,41.884,-87.6197
1,60602,1666.0,NaN,1277.0,31.1,115774.0,77279.0,32.0,2.505873,41.8835,-87.6291
2,60603,1630.0,NaN,1197.0,26.7,140114.0,106207.0,261.0,21.804511,41.8809,-87.6254
3,60604,1835.0,NaN,668.0,40.6,114565.0,130966.0,127.0,19.011976,41.8785,-87.6279
4,60605,1865.0,284200,26188.0,33.6,107811.0,81144.0,2312.0,8.828471,41.8703,-87.6236


In [12]:
chicago_housing_census_df.to_csv("Output/Chicago_Housing_and_Census_Data.csv", encoding="utf-8", index=False)

In [26]:
# Import AirBnB listing Data from Resources
# Source: http://insideairbnb.com/get-the-data.html
airbnb_df = pd.read_csv('Resources/AisBnB_listings.csv')
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,Hyde Park - Walk to UChicago/Metra to McCormic...,2613,Rebecca,NaN,Hyde Park,41.78886,-87.58671,Private room,50,2,149,5/17/2019,2.81,1,316
1,4505,We're back. 365 Great Reviews. 1 bad one.,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,117,2,368,5/5/2019,2.96,1,310
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,80,2,338,5/5/2019,2.81,1,309
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,150,4,35,5/15/2019,0.56,11,280
4,10610,3 Comforts of Cooperative Living,2140,Lois And Ed,NaN,Hyde Park,41.79708,-87.59195,Private room,35,2,38,4/12/2019,0.68,5,240


In [27]:
# Keep only required columns and create a cleaned AirBnB

airbnb_cleaned = airbnb_df[["name", "neighbourhood","latitude", "longitude", "number_of_reviews"]]
airbnb_cleaned.head()

,name,neighbourhood,latitude,longitude,number_of_reviews
0,Hyde Park - Walk to UChicago/Metra to McCormic...,Hyde Park,41.78886,-87.58671,149
1,We're back. 365 Great Reviews. 1 bad one.,South Lawndale,41.85495,-87.69696,368
2,Tiny Studio Apartment 94 Walk Score,West Town,41.90289,-87.68182,338
3,Barbara's Hideaway - Old Town,Lincoln Park,41.91769,-87.63788,35
4,3 Comforts of Cooperative Living,Hyde Park,41.79708,-87.59195,38


In [37]:
# All the latitude and longitudes need to be coverted into zipcodes.

results = []
#for every row in df_crime_latlon
for index, row in airbnb_cleaned.iterrows():
    #part of uszipcode library it takes the lat/lon from the row you are in and looks for the nearest zip (returns=1) 
    #in a 3 mile radius
    result = search.by_coordinates(row["latitude"], row["longitude"], radius=3, returns=1)
    #append to res
    results.append(result)

In [38]:
len(results)

8169

In [39]:
#create a blank list for zipcodes
zipcodes_airbnb= []
#for every entry returned in res
for i in range(len(results)):
    #this is a list of lists so for every entry there is another list inside that contains geographical information
    for x in results[i]:
        # zipcode is from uszipcode library and find the zipcode in the list
        zip = x.zipcode
     #   first = res[i]
        #append the zipcode to the zipcodes list
        zipcodes_airbnb.append(zip)
#     zipcodes.append(first)
len(zipcodes_airbnb)
zipcodes_airbnb

['60637',
 '60608',
 '60622',
 '60614',
 '60615',
 '60614',
 '60614',
 '60642',
 '60614',
 '60614',
 '60614',
 '60647',
 '60640',
 '60654',
 '60614',
 '60614',
 '60614',
 '60614',
 '60622',
 '60640',
 '60622',
 '60640',
 '60640',
 '60622',
 '60657',
 '60618',
 '60614',
 '60657',
 '60657',
 '60657',
 '60642',
 '60625',
 '60641',
 '60647',
 '60622',
 '60618',
 '60628',
 '60642',
 '60657',
 '60640',
 '60641',
 '60614',
 '60613',
 '60614',
 '60661',
 '60640',
 '60605',
 '60645',
 '60610',
 '60661',
 '60625',
 '60647',
 '60619',
 '60654',
 '60660',
 '60647',
 '60616',
 '60637',
 '60616',
 '60625',
 '60614',
 '60615',
 '60615',
 '60657',
 '60642',
 '60616',
 '60626',
 '60642',
 '60647',
 '60642',
 '60622',
 '60605',
 '60647',
 '60608',
 '60647',
 '60640',
 '60642',
 '60626',
 '60625',
 '60614',
 '60613',
 '60647',
 '60653',
 '60613',
 '60642',
 '60642',
 '60657',
 '60642',
 '60642',
 '60626',
 '60657',
 '60642',
 '60642',
 '60611',
 '60646',
 '60646',
 '60657',
 '60613',
 '60659',
 '60622',


In [40]:
# Create a dataframe of all the zipcodes 

airbnb_zip_df = pd.DataFrame({"Zipcode": zipcodes_airbnb})

In [41]:
# Join zipcode dataframe with the cleaned AirBnB dataframe

airbnb_data_by_zipcode = airbnb_cleaned.join(airbnb_zip_df)
airbnb_data_by_zipcode.head()

,name,neighbourhood,latitude,longitude,number_of_reviews,Zipcode
0,Hyde Park - Walk to UChicago/Metra to McCormic...,Hyde Park,41.78886,-87.58671,149,60637
1,We're back. 365 Great Reviews. 1 bad one.,South Lawndale,41.85495,-87.69696,368,60608
2,Tiny Studio Apartment 94 Walk Score,West Town,41.90289,-87.68182,338,60622
3,Barbara's Hideaway - Old Town,Lincoln Park,41.91769,-87.63788,35,60614
4,3 Comforts of Cooperative Living,Hyde Park,41.79708,-87.59195,38,60615


In [42]:
# Keep only required columns: Zipcode and the number of reviews for the zipcode

airbnb_zipcode = airbnb_data_by_zipcode[["Zipcode", "number_of_reviews"]]
airbnb_zipcode.head()

,Zipcode,number_of_reviews
0,60637,149
1,60608,368
2,60622,338
3,60614,35
4,60615,38


In [43]:
# Sort the dataframe by zipcode 

airbnb_zipcode_sort = airbnb_zipcode.sort_values("Zipcode")
airbnb_zipcode_sort.head()

,Zipcode,number_of_reviews
4782,46327,2
8006,60202,0
4428,60302,11
1091,60302,83
4555,60302,16


In [44]:
# Create a list of unique zipcode to visualize

airbnb_zipcode_sort["Zipcode"].unique()

array(['46327', '60202', '60302', '60304', '60456', '60501', '60601',
       '60602', '60603', '60604', '60605', '60606', '60607', '60608',
       '60609', '60610', '60611', '60612', '60613', '60614', '60615',
       '60616', '60617', '60618', '60619', '60620', '60621', '60622',
       '60623', '60624', '60625', '60626', '60628', '60629', '60630',
       '60631', '60632', '60633', '60634', '60636', '60637', '60638',
       '60639', '60640', '60641', '60642', '60643', '60644', '60645',
       '60646', '60647', '60649', '60651', '60652', '60653', '60654',
       '60656', '60657', '60659', '60660', '60661', '60707', '60712',
       '60805'], dtype=object)

In [45]:
# group the data by zipcode

grouped_zipcode = airbnb_zipcode_sort.groupby(['Zipcode'])

In [46]:
# The average review counts are the number of reviews in that zipcodes
avg_reviews = round(grouped_zipcode.mean()["number_of_reviews"])

# Create a dataframe of review count for each zipcode in the list
airbnb_reviews_zipcode = pd.DataFrame({"AirBnB Review Count": avg_reviews})
airbnb_reviews_zipcode.head()

,AirBnB Review Count
Zipcode,
46327,2.0
60202,0.0
60302,37.0
60304,1.0
60456,2.0


In [47]:
airbnb_reviews_zipcode.to_csv("Output/AirBnB_Reviews_Zipcode.csv", encoding="utf-8")

In [48]:
airbnb_zipcode_df = pd.read_csv("Output/AirBnB_Reviews_Zipcode.csv")

In [49]:
# merge AirBnB data and housing rent data on zipcode

airbnb_rent_df = pd.merge(airbnb_zipcode_df, rent_df, on="Zipcode")
airbnb_rent_df
# len(airbnb_rent_df)

,Zipcode,AirBnB Review Count,Median Rent
0,60601,11.0,1944.0
1,60602,19.0,1666.0
2,60603,10.0,1630.0
3,60604,22.0,1835.0
4,60605,20.0,1865.0
5,60606,27.0,1910.0
6,60607,50.0,1937.0
7,60608,46.0,1698.0
8,60609,25.0,1455.0
9,60610,31.0,1874.0


In [50]:
# save the csv file
airbnb_rent_df.to_csv("Output/AirBnB_Reviews_Rent_Data.csv", encoding="utf-8", index=False)